# Preprocessing

Здесь написал алгоритм чтения, feature engineering и сохранения тренировочных и тестовых данных в чанки. 

Главная идея feature engineering заключается в кодировании категориальных переменных (которыми являются все данные) и агрегировании по id и сумме. Таким образом каждое значение каждого признака приобретает силу, и чем чаще исторически клиент имел такой параметр, тем сильнее становится значение этого признака. 

Тестовые данные записыватся в единый чанк.

In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

## Анализ данных

Посмотрим на данные, изучим есть ли в них пропуски и какие категории могут встречаться. Прочтем первый parquet файл и посмотрим его структуру.

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_parquet('data/train_data/train_data_0.pq')
df.head(15)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
5,0,6,5,0,11,8,12,11,4,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,3,4,3,3,3,4,1,2,3,1,0,1
6,0,7,3,9,1,2,12,14,15,5,3,0,2,3,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
7,0,8,2,9,2,3,12,14,15,5,3,0,2,13,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
8,0,9,1,9,11,13,14,8,2,5,1,0,2,11,6,16,5,4,8,1,1,1,1,1,1,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0
9,0,10,7,9,2,10,8,8,16,4,2,0,2,11,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0


Структура исходных данных такова: по строкам записаны кредитные продукты которыми воспользовался клиент. Все значения признаков являются непорядковыми категориальными. Можно заметить, что для одного и того-же признака у одного клиента могут встречаться несколько таких значений, что как-бы усиливает значение этого признака. 

Проверим для начала наличие пропусков и дубликатов в датафрейме:

In [4]:
df[df.drop(['id', 'rn'], axis=1).duplicated()].head(10)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
86,10,16,1,3,4,3,15,14,4,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,0,0
87,10,17,1,3,4,3,15,14,4,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,0,0
88,10,18,1,3,4,3,15,14,4,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,0,0
156,20,5,5,17,4,3,5,7,4,2,3,0,2,6,6,16,5,4,8,1,0,1,1,1,16,2,17,1,1,1,1,1,0,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,0,0
158,20,7,5,17,4,3,5,7,4,2,3,0,2,6,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,0,0
498,62,4,12,9,4,8,1,11,16,3,2,0,2,9,6,16,5,4,8,1,1,1,1,1,18,2,17,0,1,1,0,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,1,1
524,68,1,6,9,4,8,1,11,8,3,3,0,2,7,6,16,5,4,8,1,1,1,1,1,4,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
559,70,24,19,5,4,8,1,11,18,2,3,0,2,6,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,1,1
561,70,26,19,5,4,8,1,11,18,2,3,0,2,6,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,1,1
567,70,32,12,0,4,8,1,11,18,2,3,0,2,6,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,5,1,1,1


Дубликаты встречаются, и хорошо бы от них избавиться. Добавим это в обработку данных.

In [5]:
path_to_dataset = 'data'

dataset_paths = sorted([os.path.join(path_to_dataset, 'train_data', filename)
                        for filename in os.listdir(os.path.join(path_to_dataset, 'train_data'))
                        if filename.startswith('train')])
for step, chunk_path in enumerate(dataset_paths):
    df = pd.read_parquet(chunk_path, columns=None)
    print(f'{step}. chunk_path {chunk_path}\n'
          f'Memory usage: {df.memory_usage().sum() / 1024**3:0.2f} Gb\n'
          f'Кол-во дубликатов: {100 * len(df[df.drop(["id", "rn"], axis=1).duplicated()])/len(df):4.2f}%\n'
          f'Кол-во пропусков: {df.isna().sum().sum()}\n')

0. chunk_path data/train_data/train_data_0.pq
Memory usage: 0.90 Gb
Кол-во дубликатов: 25.95%
Кол-во пропусков: 0

1. chunk_path data/train_data/train_data_1.pq
Memory usage: 0.96 Gb
Кол-во дубликатов: 26.66%
Кол-во пропусков: 0

2. chunk_path data/train_data/train_data_10.pq
Memory usage: 1.04 Gb
Кол-во дубликатов: 26.66%
Кол-во пропусков: 0

3. chunk_path data/train_data/train_data_11.pq
Memory usage: 1.11 Gb
Кол-во дубликатов: 28.30%
Кол-во пропусков: 0

4. chunk_path data/train_data/train_data_2.pq
Memory usage: 0.95 Gb
Кол-во дубликатов: 25.67%
Кол-во пропусков: 0

5. chunk_path data/train_data/train_data_3.pq
Memory usage: 0.96 Gb
Кол-во дубликатов: 25.58%
Кол-во пропусков: 0

6. chunk_path data/train_data/train_data_4.pq
Memory usage: 0.94 Gb
Кол-во дубликатов: 25.24%
Кол-во пропусков: 0

7. chunk_path data/train_data/train_data_5.pq
Memory usage: 0.98 Gb
Кол-во дубликатов: 26.00%
Кол-во пропусков: 0

8. chunk_path data/train_data/train_data_6.pq
Memory usage: 0.99 Gb
Кол-во дуб

**Идею преобразования данных можно выразить в таком виде:**

1. Кодируем категориальные переменные с помощью OneHotEncoder (ohe)
2. Агрегируем по id и складываем значения закодированных признаков (agg)
3. Джойним с таблицей таргетов (merge)
4. Сохраняем в новом чанке

![](pictures/image_1.png)

Таким образом значения признаков будут усиливаться для каждого клиента путем количественного сложения.

Здесь может возникнуть проблема: что делать, если мы хотим собрать все обработанные чанки в один датафрейм? Дело в том, что в разных чанках может быть разный диапазон признаков, и после кодирования колонки могут быть другими:

![](pictures/image_2.png)
![](pictures/image_3.png)

Здесь для первого клиента значение 0 в признаке pre_till_pclose_11 отвечает на вопрос "Встречалось ли когда-нибудь значение 11 для первого клиента?" ответом "Нет, не встречалось".

Когда в первом чанке кодировался столбец pre_till_pclose в нем были найдем значения: 9, 12, 16, и никогда не встречалось значение 11, а значит добавление столбца pre_till_pclose_11 с нулевыми значениями к первому чанку не изменит данные, но позволит согласовать два датафрейма с разными колонками:

![](pictures/image_4.png)

Для клиентов из первого датафрейма значения во всех новых признаках равны 0, значит такие значения там не встречались. Аналогично для второго датафрейма.

## Обработка данных

Напишем пайплайн, который загружает, очищает от дупликатов, кодирует признаки, агрегирует по id, соединяет с целевой переменной и сохраняет в новых чанках данные.

In [6]:
path_to_dataset = 'data'
max_categories = 40

if not os.path.exists(os.path.join(path_to_dataset, 'processed_data')):
    os.makedirs(os.path.join(path_to_dataset, 'processed_data'))

test = []

dataset_paths = sorted([os.path.join(path_to_dataset, 'train_data', filename)
                        for filename in os.listdir(os.path.join(path_to_dataset, 'train_data'))
                        if filename.startswith('train')])
for step, chunk_path in enumerate(dataset_paths):
    print(f'{step}. chunk_path {chunk_path}')

    # прочитать один файл с данными
    transactions_frame = pd.read_parquet(chunk_path, columns=None)
    
    # удалим дубликаты
    transactions_frame = transactions_frame.drop_duplicates()

    # закодируем данные
    columns_to_encode = transactions_frame.drop(['id', 'rn'], axis=1).columns
    ohe = OneHotEncoder(sparse_output=False, max_categories=max_categories, drop='first')
    encoded_frame = ohe.fit_transform(transactions_frame[columns_to_encode])

    # объединим закодированные категории с id
    data_preprocessed = pd.concat([transactions_frame['id'],
                                   pd.DataFrame(encoded_frame, columns=ohe.get_feature_names_out())], axis=1)
    del transactions_frame, encoded_frame

    # агрегируем по id: суммируем закодированные фичи по каждому клиенту
    data_preprocessed = data_preprocessed.groupby('id', as_index=False).agg('sum')

    # смерджим с целевой переменной flag
    targets = pd.read_csv(os.path.join(path_to_dataset, 'train_target', 'train_target.csv'))
    data_preprocessed = data_preprocessed.merge(targets, how='inner', on='id')
    del targets

    # подготовим код чанка
    block_as_str = str(step)
    if len(block_as_str) == 1:
        block_as_str = '00' + block_as_str
    else:
        block_as_str = '0' + block_as_str

    data_preprocessed_train, data_preprocessed_test = train_test_split(data_preprocessed,
                                                                       train_size=0.8,
                                                                       random_state=12,
                                                                       stratify=data_preprocessed['flag'])

    # сохраним обработанный блок в train и test
    data_preprocessed_train.to_parquet(os.path.join(path_to_dataset,
                                                    f'processed_data/train/processed_chunk_{block_as_str}.parquet'))
    print(f'Saved {path_to_dataset}/processed_data/train/processed_chunk_{block_as_str}.parquet')

    # добавим data_preprocessed_test в список test
    test.append(data_preprocessed_test)

test_df = pd.concat(test, axis=0).fillna(0.)
test_df.to_parquet(os.path.join(path_to_dataset, 'processed_data/test/test.parquet'))
print(f'Saved test.parquet')

0. chunk_path data/train_data/train_data_0.pq
Saved data/processed_data/train/processed_chunk_000.parquet
1. chunk_path data/train_data/train_data_1.pq
Saved data/processed_data/train/processed_chunk_001.parquet
2. chunk_path data/train_data/train_data_10.pq
Saved data/processed_data/train/processed_chunk_002.parquet
3. chunk_path data/train_data/train_data_11.pq
Saved data/processed_data/train/processed_chunk_003.parquet
4. chunk_path data/train_data/train_data_2.pq
Saved data/processed_data/train/processed_chunk_004.parquet
5. chunk_path data/train_data/train_data_3.pq
Saved data/processed_data/train/processed_chunk_005.parquet
6. chunk_path data/train_data/train_data_4.pq
Saved data/processed_data/train/processed_chunk_006.parquet
7. chunk_path data/train_data/train_data_5.pq
Saved data/processed_data/train/processed_chunk_007.parquet
8. chunk_path data/train_data/train_data_6.pq
Saved data/processed_data/train/processed_chunk_008.parquet
9. chunk_path data/train_data/train_data_7.p